<a href="https://colab.research.google.com/github/knuckle-walking/DataScience100/blob/main/preprocess_knock_Python_1%E5%91%A8%E7%9B%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


def get_df(filename):
  path = '/content/drive/My Drive/100knocks-preprocess/docker/work/data'
  return pd.read_csv(os.path.join(path, filename))

df_customer = get_df('customer.csv')
df_category = get_df('category.csv')
df_geocode = get_df('geocode.csv')
df_product = get_df('product.csv')
df_receipt = get_df('receipt.csv')
df_store = get_df('store.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename({'sales_ymd':'sales_date'}, axis=1).head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df_receipt[df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]['customer_id']=="CS018205000001"]

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
36,20180911,1536624000,S13018,1122,2,CS018205000001,P071401012,1,2200
9843,20180414,1523664000,S13018,1142,2,CS018205000001,P060104007,6,600
21110,20170614,1497398400,S13018,1112,2,CS018205000001,P050206001,5,990
27673,20170614,1497398400,S13018,1112,1,CS018205000001,P060702015,1,108
27840,20190216,1550275200,S13018,1152,2,CS018205000001,P071005024,1,102
28757,20180414,1523664000,S13018,1142,1,CS018205000001,P071101002,1,278
39256,20190226,1551139200,S13018,1132,2,CS018205000001,P070902035,1,168
58121,20190924,1569283200,S13018,1102,1,CS018205000001,P060805001,1,495
68117,20190226,1551139200,S13018,1132,1,CS018205000001,P071401020,1,2200
72254,20180911,1536624000,S13018,1122,1,CS018205000001,P071401005,1,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and amount >= 1000')
# .query()の中身は全部''で囲む。
# "CS018205000001"はtypeがstrだから""で囲むが、1000はtypeがintだから囲んではいけない。
# カラム名は""で囲まない。

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].query('(customer_id == "CS018205000001") and (amount >= 1000 or quantity >= 5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id=="CS018205000001" and 1000<=amount<=2000')

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id=="CS018205000001" and product_cd != "P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('not(prefecture_cd == "13") and not(floor_area > 900)')    # ドモルガン

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
23,S13008,成城店,13,東京都,東京都世田谷区成城三丁目,トウキョウトセタガヤクセイジョウサンチョウメ,03-0123-4012,139.5966,35.63614,883.0
24,S13020,十条仲原店,13,東京都,東京都北区十条仲原三丁目,トウキョウトキタクジュウジョウナカハラサンチョウメ,03-0123-4019,139.7186,35.76686,801.0
25,S13043,南品川店,13,東京都,東京都品川区南品川三丁目,トウキョウトシナガワクミナミシナガワサンチョウメ,03-0123-4027,139.7436,35.60981,845.0
29,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.4235,35.63787,810.0
30,S13017,高円寺南店,13,東京都,東京都杉並区高円寺南四丁目,トウキョウトスギナミクコウエンジミナミヨンチョウメ,03-0123-4016,139.6513,35.70273,841.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0
50,S13037,南砂一丁目店,13,東京都,東京都江東区南砂一丁目,トウキョウトコウトウクミナミスナイッチョウメ,03-0123-4023,139.8215,35.67898,814.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_store.query("store_cd.str.startswith('S14')", engine='python').head(10)
# pandas.Seriesの文字列メソッドstr.startswith()を使うと、要素が特定の文字列で始まるとTrueとなるpandas.Seriesを取得できる。
# 'engine = 'python''について、query の引数である engine には'python'か、'numexpr'かを選択することができますが、strを用いる場合は、'python'を指定してあげないとエラーが発生してしまいます。

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('customer_id.str.endswith("1")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
df_store.query('address.str.contains("横浜市")', engine="python")
# 'contains(<文字列>)'は、指定した文字列が含まれているどうかを判定する関数であり、含まれる場合はTrue、含まれない場合はFalseを返します。

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[['status_cd']]

,status_cd
0,0-00000000-0
1,0-00000000-0
2,D-20100325-C
3,0-00000000-0
4,6-20090929-2
...,...
21966,0-00000000-0
21967,F-20101028-F
21968,0-00000000-0
21969,0-00000000-0


In [ ]:
df_customer.query('status_cd.str.contains("^[A-F]", regex=True)', engine='python').head(10)
# ^○○：直後の文字列が○○から始まる
# [A-F]：AからFのうち１つ
# [A-F]{k}：AからFのうち1つがk個
# regex=True：特殊文字を含む（今回の場合："ー"）．regexはcontainsだけであり，startswithはもってない．
# containsでstartswithと同じ処理をするために，正規表現として^を入れている．

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('status_cd.str.contains("[1-9]$", regex=True)', engine='python').head(10)
# ○○＄：○○で終わるを意味する正規表現
# regex=True：str.containsの第2引数として入れる．

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4



---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query('status_cd.str.contains("^[A-F].*[1-9]$", regex=True)', engine="python").head(10)
# .：任意の1文字
# *：直前が0以上繰り返す
# .*：任意の文字が0回以上繰り返す
# ^[A-F].*[1-9]$：[A-F]のうちの1つで始まり，[1-9]のうちの1つで終わり，それらの間に0以上の任意の文字がある．

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
# df_store[['tel_no']]
df_store.query('tel_no.str.contains("^[0-9]{3}-[0-9]{3}-[0-9]{4}$", regex=True)', engine='python')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-017: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day', ascending=True).head(10)
# df.sort_values("カラム名")：カラム名の要素でソートをかける

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-018: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day', ascending=True).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-019: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_receipt[['ranking']] = df_receipt[['amount']].rank(method='min', ascending=False)
# rank()メソッドを呼ぶと，デフォルトでは各列が昇順で順位付けされる．
# 同一値（重複値）は平均順位となり，文字列はアルファベット順に比較される．
# 引数: method：同一値（重複値）の処理を指定
# .rank(, ascending=False)：値が大きいものに小さな値のランキングをつける．（(売上金額(amount)が高い順に)対応する）

df_tmp = pd.concat([df_receipt[['customer_id', 'amount']], df_receipt[['ranking']]], axis=1)
# axis：0のときは縦方向．1のときは横方向．

df_tmp.sort_values('ranking', ascending=True).head(10)
# sort_valuesの引数は'カラム名'

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_receipt[['ranking']] = df_receipt[['amount']].rank(method='first', ascending=False)
# method='first'：同一値（重複値）は登場順に順位付けされる．
# method='first'：数値のみに有効なので注意。文字列が含まれている場合はnumeric_only=Trueとする。
# ascending=False：金額が大きいものに1位をつける．

df_tmp = pd.concat([df_receipt[['customer_id', 'amount']], df_receipt[['ranking']]], axis=1)
df_tmp.sort_values('ranking', ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
len(df_receipt['customer_id'].unique())    # unique：数値に対して適用可能．dfに対しては不可

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':'sum',
                                    'quantity':'sum'}).reset_index()
# groupby は、同じ値を持つデータをまとめて、それぞれの塊に対して共通の操作を行いたい時に使う。
# reset_index：インデックスを連番に振り直し

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head(10)
# groupby('XX')：XXごとに操作をする．
# groupby('XX').'カラム名'.max()：同一カラム名(XX)のうち，'カラム名'の最大値を取得

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').sales_ymd.min().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_tmp.columns

Index(['customer_id', 'amount', 'ranking'], dtype='object')

In [ ]:
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_tmp.columns = ['customer_id', 'new_sales_ymd', 'old_sales_ymd']
df_tmp.query('new_sales_ymd != old_sales_ymd').head(10)

,customer_id,new_sales_ymd,old_sales_ymd
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_tmp = df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index()    # なぜreset_index()をするのかわからない．
df_tmp.sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df_tmp = df_receipt.groupby('store_cd').agg({'amount':'median'}).reset_index()
df_tmp.sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [ ]:
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index()
# .agg({'XX':'mode'})は存在しない
# map, applyを利用することで，自作関数をdfに対して適用可能．

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001
5,S13001,0,P060303001
6,S13002,0,P060303001
7,S13003,0,P071401001
8,S13004,0,P060303001
9,S13005,0,P040503001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.apply(lambda x: x.var(ddof=0)).reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.apply(lambda x: x.std(ddof=0)).reset_index().sort_values('amount', ascending=True).head()

,store_cd,amount
51,S14050,397.112239
45,S14042,414.802588
50,S14049,424.690284
4,S12030,430.746907
26,S13044,435.428471


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
# 解1
np.percentile(df_receipt['amount'], [25, 50, 75, 100])

array([  102.,   170.,   288., 10925.])

In [ ]:
# 別解
df_receipt['amount'].quantile(q=[0.25, 0.50, 0.75, 0.100])

0.25    102.0
0.50    170.0
0.75    288.0
0.10     88.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_tmp = df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index()
df_tmp.query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
df_tmp = df_receipt.query('customer_id.str.contains("^[A-Y]", regex=True)', engine='python')
df_tmp.groupby('customer_id').agg({'amount':'sum'}).agg({'amount':'mean'})

amount    2547.742235
dtype: float64

In [ ]:
# 別解1 aggを使わない
df_receipt.query('customer_id.str.contains("^[A-Y]", regex=True)', engine='python').groupby('customer_id').amount.sum().mean()

2547.742234529256

In [ ]:
# 別解2 queryを使わない
df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().mean()

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [ ]:
df_mean_tmp = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').amount.sum().mean()
df_sum_tmp = df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').amount.sum().reset_index()
df_sum_tmp[df_sum_tmp['amount']>=df_mean_tmp].head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
pd.merge(df_receipt, df_store[['store_cd','store_name']], how='inner', on='store_cd').head(10)    # how='inner'：内部結合
# 内部結合(inner join)：左右のDataFrameでお互い一致するデータを残す結合方法。A∩B
# 左結合(left join)：左のDataFrameに一致するデータを残す結合方法。Aのみ：A∪(A∩B)
# 右結合(right join)：右のDataFrameに一致するデータを残す結合方法。Bのみ：B∪(A∩B)
# 外部結合(outer join)：左右両方のDataFrameのデータを残す結合方法。A∪B

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,ranking,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,55059.0,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,102208.0,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,38472.0,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,95568.0,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,57042.0,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,91292.0,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,95569.0,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,15936.0,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,20465.0,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,73244.0,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [ ]:
pd.merge(df_product, df_category[['category_major_cd', 'category_small_name']], on='category_major_cd', how='inner').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
1,P040101001,4,401,40101,198.0,149.0,寿司類
2,P040101001,4,401,40101,198.0,149.0,魚介佃煮類
3,P040101001,4,401,40101,198.0,149.0,海草佃煮類
4,P040101001,4,401,40101,198.0,149.0,野菜佃煮類
5,P040101001,4,401,40101,198.0,149.0,豆佃煮類
6,P040101001,4,401,40101,198.0,149.0,サラダ類
7,P040101001,4,401,40101,198.0,149.0,炒め・煮物類
8,P040101001,4,401,40101,198.0,149.0,和え物
9,P040101001,4,401,40101,198.0,149.0,酢の物


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp = df_customer.query('gender_cd == 1 and not customer_id.str.contains("^[Z]", regex=True)', engine='python')
pd.merge(df_tmp[['customer_id']], df_amount_sum, on='customer_id', how='left').fillna(0).head(10)    # 主軸は"customer_id"

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
# ("Z"から始まる顧客を除外)
df_tmp = df_receipt.query('not customer_id.str.contains("^[Z]", regex=True)', engine='python')

# 売上金額合計の多い顧客TOP20
df_receipt_amount = df_tmp.groupby('customer_id').amount.sum().reset_index().sort_values('amount', ascending=False).head(20)

# 売上日数の多い顧客のTOP20
df_receipt_ymd = df_tmp.drop_duplicates(subset=['customer_id', 'sales_ymd']).groupby('customer_id').sales_ymd.count().reset_index().sort_values('sales_ymd', ascending=False).head(20)
# drop_duplicates(subset=['customer_id', 'sales_ymd'])：同じ日に複数個購入したcustomer_idは1つとしてカウントする．

# 結合
pd.merge(df_receipt_amount, df_receipt_ymd, on='customer_id', how='outer')

,customer_id,amount,sales_ymd
0,CS017415000097,23086.0,20.0
1,CS015415000185,20153.0,22.0
2,CS031414000051,19202.0,19.0
3,CS028415000007,19127.0,21.0
4,CS001605000009,18925.0,NaN
5,CS010214000010,18585.0,22.0
6,CS016415000141,18372.0,20.0
7,CS006515000023,18372.0,NaN
8,CS011414000106,18338.0,NaN
9,CS038415000104,17847.0,NaN




---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [ ]:
store_tmp = len(df_store[['store_cd']].drop_duplicates(subset=['store_cd']))
product_tmp = len(df_product[['product_cd']].drop_duplicates(subset=['product_cd']))

store_tmp * product_tmp

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [ ]:
df_sales_amount_by_date = df_receipt.groupby('sales_ymd').amount.sum().reset_index().sort_values('sales_ymd', ascending=True)

# 階差を算出する関数を定義
def diff(df):
  # 階差を格納
  diff = []
  # 階差を求める
  for i in range(len(df)):
    try:
      diff.append(df[i] - df[i-1])
    except:
      diff.append(np.nan)
  return diff

# 元のdfに"diff"列を追加
df_sales_amount_by_date[['diff']] = pd.DataFrame(diff(df_sales_amount_by_date['amount']))
df_sales_amount_by_date

,sales_ymd,amount,diff
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0
...,...,...,...
1029,20191027,37484,-14287.0
1030,20191028,40161,2677.0
1031,20191029,36091,-4070.0
1032,20191030,26602,-9489.0


In [ ]:
# 別解

df_sales_amount_by_date = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_sales_amount_by_date = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift(periods=1)], axis=1) # shift：元のdfをperiods分だけ移動
# 結合のmergeとconcatの比較
## merge：2つのdfを横方向に結合
## concat：その他

df_sales_amount_by_date.columns = ['sales_ymd', 'amount', 'lag_ymd', 'lag_amount']
df_sales_amount_by_date['diff_amount'] = df_sales_amount_by_date['amount'] - df_sales_amount_by_date['lag_amount']
df_sales_amount_by_date



,sales_ymd,amount,lag_ymd,lag_amount,diff_amount
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,-9558.0
2,20170103,27503,20170102.0,24165.0,3338.0
3,20170104,36165,20170103.0,27503.0,8662.0
4,20170105,37830,20170104.0,36165.0,1665.0
...,...,...,...,...,...
1029,20191027,37484,20191026.0,51771.0,-14287.0
1030,20191028,40161,20191027.0,37484.0,2677.0
1031,20191029,36091,20191028.0,40161.0,-4070.0
1032,20191030,26602,20191029.0,36091.0,-9489.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [ ]:
# 別解1
df_sales_amount_by_date = df_receipt.groupby('sales_ymd').sum().reset_index()


In [ ]:
df_sales_amount_by_date = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_lag1 = df_sales_amount_by_date.shift(periods=1).rename(columns={'sales_ymd':'lag1_ymd', 'amount':'lag1_amount'})
df_lag2 = df_sales_amount_by_date.shift(periods=2).rename(columns={'sales_ymd':'lag2_ymd', 'amount':'lag2_amount'})
df_lag3 = df_sales_amount_by_date.shift(periods=3).rename(columns={'sales_ymd':'lag3_ymd', 'amount':'lag3_amount'})

df_sales_amount_by_date = pd.concat([df_sales_amount_by_date, df_lag1, df_lag2, df_lag3], axis=1)
df_sales_amount_by_date.dropna().head(10)

,sales_ymd,amount,lag1_ymd,lag1_amount,lag2_ymd,lag2_amount,lag3_ymd,lag3_amount
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0
10,20170111,24287,20170110.0,20143.0,20170109.0,26718.0,20170108.0,24737.0
11,20170112,23526,20170111.0,24287.0,20170110.0,20143.0,20170109.0,26718.0
12,20170113,28004,20170112.0,23526.0,20170111.0,24287.0,20170110.0,20143.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_sales_summary = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_sales_summary['era'] = (df_sales_summary['age']/ 10).astype('int') * 10
df_sales_summary = pd.pivot_table(df_sales_summary, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index().rename(columns={0:'male', 1:'female', 9:'unknown'})
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [ ]:
df_sales_summary.set_index('era').stack().reset_index().replace({'male':'00', 'female':'01', 'unknown':'99'}).rename(columns={'level_1':'gender_cd', 0:'amount'})
# df.set_index('XX')：インデックスとしてXXを採用する
# df.stack()：インデックスごとのdfをつくる

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')], axis=1).head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711



---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['application_date'])], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,1970-01-01 00:00:00.020150905
1,CS037613000071,1970-01-01 00:00:00.020150414
2,CS031415000172,1970-01-01 00:00:00.020150529
3,CS028811000001,1970-01-01 00:00:00.020160115
4,CS001215000145,1970-01-01 00:00:00.020170605
5,CS020401000016,1970-01-01 00:00:00.020150225
6,CS015414000103,1970-01-01 00:00:00.020150722
7,CS029403000008,1970-01-01 00:00:00.020150515
8,CS015804000004,1970-01-01 00:00:00.020150607
9,CS033513000180,1970-01-01 00:00:00.020150728


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no']], df_receipt[['receipt_sub_no']], pd.to_datetime(df_receipt['sales_ymd'], format='%Y%m%d')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
# UNIX時間（エポック秒）：協定世界時 (UTC) での1970年1月1日午前0時0分0秒からの経過秒数。

pd.concat([df_receipt[['receipt_no']], df_receipt[['receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s')], axis=1).head(10)
# デフォルトでunit='ms'つまり、ミリ秒であるが、今回は秒で与えられているのでunit='s'とした？

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt['receipt_no'], df_receipt['receipt_sub_no'], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.year], axis=1).head(10)
# df.dt.year：datetime型のYearの部分だけ取り出す。

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、「月」は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt['receipt_no'], df_receipt['receipt_sub_no'], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%m')], axis=1).head(10)
# df.dt.month：1ケタで出力
# df.dt.strftime('%m')：2ケタで出力

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、「日」は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt['receipt_no'], df_receipt['receipt_sub_no'], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_receipt_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
df_receipt_tmp = pd.concat([df_receipt_tmp[['customer_id', 'amount']], df_receipt_tmp['amount'].apply(lambda x: 1 if x > 2000 else 0)], axis=1).head(10)
df_receipt_tmp.columns = ['customer_id', 'amount', 'sales_flg']
df_receipt_tmp

,customer_id,amount,sales_flg
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


In [ ]:
# 別解（concatを使わない）
df_receipt_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
df_receipt_tmp['sales_flg'] = df_receipt_tmp['amount'].apply(lambda x: 1 if x > 2000 else 0)
df_receipt_tmp.head(10)

,customer_id,amount,sales_flg
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において売上実績がある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
df_tmp = df_customer[['customer_id', 'postal_cd']].copy()
df_tmp['postal_flg'] = df_tmp['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)
pd.merge(df_receipt, df_tmp, on='customer_id', how='inner').groupby('postal_flg').agg({'customer_id':'nunique'})


,customer_id
postal_flg,
0,3906
1,4400


---
> P-054: 顧客データフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_customer[['customer_id', 'address']].copy()
df_tmp['add_num'] = df_customer['address'].str[0:3].map({'埼玉県':'11', '千葉県':'12', '東京都':'13', '神奈川':'14'})
df_tmp.head(10)

,customer_id,address,add_num
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
quantiles = df_tmp.quantile([0.25, 0.5, 0.75])
q1 = quantiles['amount'][0.25]
q2 = quantiles['amount'][0.5]
q3 = quantiles['amount'][0.75]

def filter(x):
    if x < q1:
      return 1
    elif q1 <= x < q2:
      return 2
    elif q2 <= x < q3:
      return 3
    elif q3 <= x:
      return 4

df_tmp['category'] = df_tmp['amount'].apply(lambda x: filter(x))
df_tmp.head(10)

,customer_id,amount,category
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer['era'] = df_customer['age'].apply(lambda x: np.array([10*int(x/10), 60]).min())
df_customer[['customer_id', 'birth_day', 'era']].head(10)

# df_customer['era'].apply(lambda x: 60 if x >= 60 else XXX)としてもdfの中身は変換されなかった

,customer_id,birth_day,era
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
# gender_cdが(女性, 男性, 不明)＝(１, 0, 9)であることに着目

df_customer['gender_era'] = df_customer['gender_cd'] + df_customer['era']
pd.merge(df_customer[['customer_id', 'birth_day', 'era']], df_customer[['customer_id', 'gender_era']], on='customer_id', how='inner').head(10)
# pd.concatdf_customer.head(10)

,customer_id,birth_day,era,gender_era
0,CS021313000114,1981-04-29,30,31
1,CS037613000071,1952-04-01,60,69
2,CS031415000172,1976-10-04,40,41
3,CS028811000001,1933-03-27,60,61
4,CS001215000145,1995-03-29,20,21
5,CS020401000016,1974-09-15,40,40
6,CS015414000103,1977-08-09,40,41
7,CS029403000008,1973-08-17,40,40
8,CS015804000004,1931-05-02,60,60
9,CS033513000180,1962-07-11,50,51


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)
# pd.get_dummies(df, columns=XX)：dfを渡して，カラム名XXでダミー変数を作成

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index().query('not customer_id.str.startswith("Z")', engine='python')
df_tmp['amount_ss'] = preprocessing.scale(df_tmp['amount'])
df_tmp.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


In [ ]:
# コード例2

df_tmp = df_receipt.copy().groupby('customer_id').amount.sum().reset_index().query('not customer_id.str.startswith("Z")', engine='python')
scaler = preprocessing.StandardScaler()
scaler.fit(df_tmp[['amount']])   # fitの引数は2次元で．
df_tmp['amount_ss'] = scaler.transform(df_tmp[['amount']])    # transformに再びdfを入れる
df_tmp.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


In [ ]:
# 別解　標準化の定義より
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index().query('not customer_id.str.startswith("Z")', engine='python')
avg = df_tmp['amount'].mean()
std = df_tmp['amount'].std()
df_tmp['amount_ss_'] = df_tmp['amount'].apply(lambda x: (x - avg)/std)
df_tmp.head(10)

,customer_id,amount,amount_ss_
0,CS001113000004,1298,-0.459350
1,CS001114000005,626,-0.706348
2,CS001115000010,3044,0.182403
3,CS001205000004,1988,-0.205737
4,CS001205000006,3337,0.290096
5,CS001211000025,456,-0.768832
6,CS001212000027,448,-0.771773
7,CS001212000031,296,-0.827641
8,CS001212000046,228,-0.852635
9,CS001212000070,456,-0.768832


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
scaler = preprocessing.MinMaxScaler()
scaler.fit(df_tmp[['amount']])
df_tmp['amount_ss'] = scaler.transform(df_tmp[['amount']])
df_tmp.head(10)

,customer_id,amount,amount_ss
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


In [ ]:
# 別解　定義より
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
min = df_tmp['amount'].min()
max = df_tmp['amount'].max()
df_tmp['amount_ss_'] = df_tmp['amount'].apply(lambda x: (x - min)/(max - min))
df_tmp.head(10)

,customer_id,amount,amount_ss_
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_log10'] = df_tmp['amount'].apply(lambda x: np.log10(x))
df_tmp.head(10)

,customer_id,amount,amount_log10
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
5,CS001211000025,456,2.658965
6,CS001212000027,448,2.651278
7,CS001212000031,296,2.471292
8,CS001212000046,228,2.357935
9,CS001212000070,456,2.658965


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_ln'] = df_tmp['amount'].apply(lambda x: np.log(x))
df_tmp.head(10)

,customer_id,amount,amount_ln
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
5,CS001211000025,456,6.122493
6,CS001212000027,448,6.104793
7,CS001212000031,296,5.690359
8,CS001212000046,228,5.429346
9,CS001212000070,456,6.122493


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_product.copy()
df_tmp['profit'] = df_product['unit_price'] - df_product['unit_cost']
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
0,P040101001,4,401,40101,198.0,149.0,49.0
1,P040101002,4,401,40101,218.0,164.0,54.0
2,P040101003,4,401,40101,230.0,173.0,57.0
3,P040101004,4,401,40101,248.0,186.0,62.0
4,P040101005,4,401,40101,268.0,201.0,67.0
5,P040101006,4,401,40101,298.0,224.0,74.0
6,P040101007,4,401,40101,338.0,254.0,84.0
7,P040101008,4,401,40101,420.0,315.0,105.0
8,P040101009,4,401,40101,498.0,374.0,124.0
9,P040101010,4,401,40101,580.0,435.0,145.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
# 利益率：売上に対する利益
df_tmp = df_product.copy()
df_tmp['unit_profit_rate'] = (df_tmp['unit_price'] - df_tmp['unit_cost'])/df_tmp['unit_price']
df_tmp['unit_profit_rate'].mean(skipna=True)

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
round(22/7)

3

In [ ]:
df_tmp = df_product.copy()

"""
0.3 = (price - cost)/price
でpriceを求めたい
0.7price = cost
price = cost/0.7
"""

df_tmp['new_price'] = np.floor(df_tmp['unit_cost']/0.7)
# np.ceil；切り上げ
# np.floor：切り捨て
# np.trunc：切り捨て
# np.round：四捨五入
df_tmp['conf'] = (df_tmp['new_price'] - df_tmp['unit_cost'])/df_tmp['new_price']


df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,conf
0,P040101001,4,401,40101,198.0,149.0,212.0,0.297170
1,P040101002,4,401,40101,218.0,164.0,234.0,0.299145
2,P040101003,4,401,40101,230.0,173.0,247.0,0.299595
3,P040101004,4,401,40101,248.0,186.0,265.0,0.298113
4,P040101005,4,401,40101,268.0,201.0,287.0,0.299652
5,P040101006,4,401,40101,298.0,224.0,320.0,0.300000
6,P040101007,4,401,40101,338.0,254.0,362.0,0.298343
7,P040101008,4,401,40101,420.0,315.0,450.0,0.300000
8,P040101009,4,401,40101,498.0,374.0,534.0,0.299625
9,P040101010,4,401,40101,580.0,435.0,621.0,0.299517


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()

df_tmp['new_price'] = np.round(df_tmp['unit_cost']/0.7)
df_tmp['conf'] = (df_tmp['new_price'] - df_tmp['unit_cost'])/df_tmp['new_price']

df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,conf
0,P040101001,4,401,40101,198.0,149.0,213.0,0.300469
1,P040101002,4,401,40101,218.0,164.0,234.0,0.299145
2,P040101003,4,401,40101,230.0,173.0,247.0,0.299595
3,P040101004,4,401,40101,248.0,186.0,266.0,0.300752
4,P040101005,4,401,40101,268.0,201.0,287.0,0.299652
5,P040101006,4,401,40101,298.0,224.0,320.0,0.300000
6,P040101007,4,401,40101,338.0,254.0,363.0,0.300275
7,P040101008,4,401,40101,420.0,315.0,450.0,0.300000
8,P040101009,4,401,40101,498.0,374.0,534.0,0.299625
9,P040101010,4,401,40101,580.0,435.0,621.0,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()

df_tmp['new_price'] = np.ceil(df_tmp['unit_cost']/0.7)
df_tmp['conf'] = (df_tmp['new_price'] - df_tmp['unit_cost'])/df_tmp['new_price']

df_tmp.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,conf
0,P040101001,4,401,40101,198.0,149.0,213.0,0.300469
1,P040101002,4,401,40101,218.0,164.0,235.0,0.302128
2,P040101003,4,401,40101,230.0,173.0,248.0,0.302419
3,P040101004,4,401,40101,248.0,186.0,266.0,0.300752
4,P040101005,4,401,40101,268.0,201.0,288.0,0.302083


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()

df_tmp['in_tax'] = np.floor(df_tmp['unit_price']*1.1)

df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,in_tax
0,P040101001,4,401,40101,198.0,149.0,217.0
1,P040101002,4,401,40101,218.0,164.0,239.0
2,P040101003,4,401,40101,230.0,173.0,253.0
3,P040101004,4,401,40101,248.0,186.0,272.0
4,P040101005,4,401,40101,268.0,201.0,294.0
5,P040101006,4,401,40101,298.0,224.0,327.0
6,P040101007,4,401,40101,338.0,254.0,371.0
7,P040101008,4,401,40101,420.0,315.0,462.0
8,P040101009,4,401,40101,498.0,374.0,547.0
9,P040101010,4,401,40101,580.0,435.0,638.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
df_tmp = pd.merge(df_receipt, df_product, on='product_cd', how='inner')
df_tmp1 = df_tmp.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp2 = df_tmp.query('category_major_cd == 7').groupby('customer_id').amount.sum().reset_index()

df_tmp3 = pd.merge(df_tmp1, df_tmp2, on='customer_id', how='inner')
df_tmp3['rate'] = df_tmp3['amount_y']/df_tmp3['amount_x']
df_tmp3.head(10)

,customer_id,amount_x,amount_y,rate
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
# df_receipt
df_tmp1 = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates().copy()
df_tmp1['sales_ymd'] = pd.to_datetime(df_tmp1['sales_ymd'], format='%Y%m%d')

# df_customer
df_tmp2 = df_customer[['customer_id', 'application_date']].drop_duplicates().copy()
df_tmp2['application_date'] = pd.to_datetime(df_tmp2['application_date'], format='%Y%m%d')

# 結合
df_tmp3 = pd.merge(df_tmp1, df_tmp2, on='customer_id', how="inner")

# 算出
df_tmp3['lag_days'] = df_tmp3['sales_ymd'] - df_tmp3['application_date']
df_tmp3.head(10)

,customer_id,sales_ymd,application_date,lag_days
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
3,CS006214000001,2018-10-28,2015-02-01,1365 days
4,CS006214000001,2019-09-08,2015-02-01,1680 days
5,CS006214000001,2018-01-31,2015-02-01,1095 days
6,CS006214000001,2017-07-05,2015-02-01,885 days
7,CS006214000001,2018-11-10,2015-02-01,1378 days
8,CS006214000001,2019-04-10,2015-02-01,1529 days
9,CS006214000001,2019-06-01,2015-02-01,1581 days


In [ ]:
# 別解 relativedeltaの利用

df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], on='customer_id', how='inner').drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')
df_tmp['lag_days'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years*365 + relativedelta(x[0], x[1]).months*30 + relativedelta(x[0], x[1]).days, axis=1)

df_tmp.head(10)

,customer_id,sales_ymd,application_date,lag_days
0,CS006214000001,2018-11-03,2015-02-01,1367
1,CS006214000001,2017-05-09,2015-02-01,828
2,CS006214000001,2017-06-08,2015-02-01,857
4,CS006214000001,2018-10-28,2015-02-01,1362
7,CS006214000001,2019-09-08,2015-02-01,1677
8,CS006214000001,2018-01-31,2015-02-01,1090
9,CS006214000001,2017-07-05,2015-02-01,884
10,CS006214000001,2018-11-10,2015-02-01,1374
12,CS006214000001,2019-04-10,2015-02-01,1529
15,CS006214000001,2019-06-01,2015-02-01,1580


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], on='customer_id', how='inner').drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['lag_month'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis=1)


df_tmp.head(10)

,customer_id,sales_ymd,application_date,lag_month
0,CS006214000001,2018-11-03,2015-02-01,45
1,CS006214000001,2017-05-09,2015-02-01,27
2,CS006214000001,2017-06-08,2015-02-01,28
4,CS006214000001,2018-10-28,2015-02-01,44
7,CS006214000001,2019-09-08,2015-02-01,55
8,CS006214000001,2018-01-31,2015-02-01,35
9,CS006214000001,2017-07-05,2015-02-01,29
10,CS006214000001,2018-11-10,2015-02-01,45
12,CS006214000001,2019-04-10,2015-02-01,50
15,CS006214000001,2019-06-01,2015-02-01,52


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], on='customer_id', how='inner').drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['lag_year'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years, axis=1)
df_tmp.head(10)

,customer_id,sales_ymd,application_date,lag_year
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
7,CS006214000001,2019-09-08,2015-02-01,4
8,CS006214000001,2018-01-31,2015-02-01,2
9,CS006214000001,2017-07-05,2015-02-01,2
10,CS006214000001,2018-11-10,2015-02-01,3
12,CS006214000001,2019-04-10,2015-02-01,4
15,CS006214000001,2019-06-01,2015-02-01,4


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']], on='customer_id', how='inner').drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['lag_utc'] = (df_tmp['sales_ymd'].astype(np.int64) - df_tmp['application_date'].astype(np.int64))/(10**9)
# df.astype(np.int64)ではYYYYMMDDから[ns]が出力される．よって，[s]にするために10^9でわる．

df_tmp.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """


,customer_id,sales_ymd,application_date,lag_utc
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']].copy().drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['Monday'] = df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))
# x - relaticedelta(days=x.weekday())：これによって，月曜日を取得

df_tmp['lag_from_monday'] = df_tmp[['sales_ymd', 'Monday']].apply(lambda x: relativedelta(x[0], x[1]).days, axis=1)

df_tmp.head(10)

,customer_id,sales_ymd,Monday,lag_from_monday
0,CS006214000001,2018-11-03,2018-10-29,5
1,CS008415000097,2018-11-18,2018-11-12,6
2,CS028414000014,2017-07-12,2017-07-10,2
3,ZZ000000000000,2019-02-05,2019-02-04,1
4,CS025415000050,2018-08-21,2018-08-20,1
5,CS003515000195,2019-06-05,2019-06-03,2
6,CS024514000042,2018-12-05,2018-12-03,2
7,CS040415000178,2019-09-22,2019-09-16,6
8,ZZ000000000000,2017-05-04,2017-05-01,3
9,CS027514000015,2019-10-10,2019-10-07,3


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
df_customer.sample(frac=0.01, random_state=100).head(10)

# df.sample(frac=0.01)：dfからランダムに1%を抽出

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,era,gender_era
10290,CS005412000024,杉山 まみ,1,女性,1977-01-14,42,176-0002,東京都練馬区桜台**********,S13005,20150808,2-20080407-1,40,41
7895,CS004311000083,成海 由美子,1,女性,1984-08-29,34,179-0075,東京都練馬区高松**********,S13004,20160816,0-00000000-0,30,31
8244,CS020512000211,小菅 ヒカル,1,女性,1963-10-08,55,174-0055,東京都板橋区泉町**********,S13020,20161227,0-00000000-0,50,51
19698,CS031512000163,杉浦 りえ,1,女性,1963-02-13,56,164-0002,東京都中野区上高田**********,S13031,20150625,6-20100712-5,50,51
13447,CS044413000015,本山 まひる,1,女性,1970-09-10,48,144-0044,東京都大田区本羽田**********,S13044,20170923,0-00000000-0,40,41
10620,CS040315000187,竹村 なぎさ,1,女性,1982-10-18,36,227-0064,神奈川県横浜市青葉区田奈町**********,S14040,20150810,0-00000000-0,30,31
14781,CS004603000085,大下 一哉,0,男性,1949-03-28,70,165-0033,東京都中野区若宮**********,S13004,20170323,0-00000000-0,60,60
14909,CS024313000143,黒崎 さゆり,1,女性,1981-03-31,38,216-0015,神奈川県川崎市宮前区菅生**********,S14024,20150122,0-00000000-0,30,31
16700,CS032412000045,中井 さやか,1,女性,1974-02-10,45,210-0006,神奈川県川崎市川崎区砂子**********,S13032,20160206,A-20100425-5,40,41
7675,CS022705000006,石倉 染五郎,0,男性,1941-04-16,77,249-0005,神奈川県逗子市桜山**********,S14022,20180103,0-00000000-0,60,60


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別ごとに件数を集計せよ。

In [ ]:
# 層化抽出：元のデータを同じ分布でランダムに抽出すること


df_tmp, _df_tmp = train_test_split(df_customer, train_size=0.1, stratify=df_customer['gender_cd'])
# train, test = train_test_split(array_like, train_size=0.6)：データを0.6, 0.4に分割
# train, test = train_test_split(array_like, test_size=0.6)：データを0.4, 0.6に分割
# stratify = XX：XXについて層化抽出

df_tmp.groupby('gender_cd').agg({'customer_id' : 'count'})

,customer_id
gender_cd,
0,298
1,1792
9,107


---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python')[['customer_id', 'amount']].copy()

df_tmp = df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp['3sigma'] = df_tmp['amount'].mean() + df_tmp['amount'].std()*3    # amountは金額より，-3σは考慮しなくてよい

df_tmp['error'] = df_tmp[['amount', '3sigma']].apply(lambda x: x[0] - x[1], axis=1)


df_tmp[df_tmp['error'] >= 0].head(10)

,customer_id,amount,3sigma,error
332,CS001605000009,18925,10709.76435,8215.23565
1755,CS006415000147,12723,10709.76435,2013.23565
1817,CS006515000023,18372,10709.76435,7662.23565
1833,CS006515000125,12575,10709.76435,1865.23565
1841,CS006515000209,11373,10709.76435,663.23565
1870,CS007115000006,11528,10709.76435,818.23565
1941,CS007514000056,13293,10709.76435,2583.23565
1943,CS007514000094,15735,10709.76435,5025.23565
1951,CS007515000107,11188,10709.76435,478.23565
1997,CS007615000026,11959,10709.76435,1249.23565


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
df_tmp = df_receipt.query('not customer_id.str.startswith("Z")', engine='python')[['customer_id', 'amount']].copy()
df_tmp = df_tmp.groupby('customer_id').amount.sum().reset_index()


# def thresh(/df):
quantiles = np.quantile(df_tmp['amount'], np.linspace(0, 1, 5))
Q1, Q2, Q3 = quantiles[1], quantiles[2], quantiles[3]
IQR = Q3 - Q1

th_under = Q1 - 1.5*IQR
th_upper = Q3 + 1.5*IQR

# if df <= th_under:
#   return 0
# elif df >= th_upper:
#   return 0


df_tmp = df_tmp.query('amount <= @th_under or amount >= @th_upper')

# df_tmp = df_tmp['amount'].apply(lambda x: thresh(x))


df_tmp.head(10)

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()
# isnull()：要素ごとに欠損値か判定 True or False
# sum()：Trueの分だけ数え上げ

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_product
df_product_1 = df_product.dropna(inplace=False)
# inplace=Trueにすれば変更を元のデータに反映させることが可能です。

print(f'df_product：{len(df_product)}')
print(f'df_product_1：{len(df_product_1)}')

df_product：10030
df_product_1：10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product_2 = df_product.copy()

df_product_2['unit_price'] = df_product_2['unit_price'].fillna(np.round(df_product_2['unit_price'].mean()))
df_product_2['unit_cost'] = df_product_2['unit_cost'].fillna(np.round(df_product_2['unit_cost'].mean()))



df_product_2.isnull().sum() == 0

product_cd            True
category_major_cd     True
category_medium_cd    True
category_small_cd     True
unit_price            True
unit_cost             True
dtype: bool

In [ ]:
# 別解　同時にfillna

df_product_2 = df_product.fillna({
    'unit_price': np.round(df_product['unit_price'].mean()),
    'unit_cost': np.round(df_product['unit_cost'].mean())})

df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product_3 = df_product.fillna({
    'unit_price':np.round(df_product['unit_price'].median()),
    'unit_cost':np.round(df_product['unit_cost'].median())
})

df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# unit_priceとunit_costの中央値を取得
df_tmp = df_product.groupby('category_small_cd').agg({'unit_price':'median', 'unit_cost':'median'}).reset_index()
df_tmp.columns = ['category_small_cd', 'median_price', 'median_cost']

# 内部結合する
df_product_4 = pd.merge(df_product, df_tmp, on='category_small_cd', how='inner')

# unit_priceとunit_costのうちnanにはmedianの値を格納する．
df_product_4['unit_price'] = df_product_4[['unit_price', 'median_price']].apply(lambda x: x[1] if np.isnan(x[0]) else x[0], axis=1)
df_product_4['unit_cost'] = df_product_4[['unit_cost', 'median_cost']].apply(lambda x: x[1] if np.isnan(x[0]) else x[0], axis=1)

# 欠損値が0であることを確認
df_product_4.isnull().sum()


product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
median_price          0
median_cost           0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [ ]:
# 該当期間のみdf
df_tmp1 = df_receipt.query('20190101 <= sales_ymd <= 20191231')

# 該当期間の顧客ごとのamount合計
df_tmp1 = df_tmp1.groupby('customer_id').agg({'amount':'sum'}).reset_index().rename(columns={'amount':'amount_2019'})

# 全期間の顧客ごとのamount合計
df_tmp2 = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index().rename(columns={'amount':'amount_all'})

# 解答ではhow='outer'とあったが，自分の考え方と沿うようにhow='inner'とし，dropna()で帳尻を合わせた．
df_tmp = pd.merge(df_tmp1, df_tmp2, on='customer_id', how='right').dropna()

df_tmp['amount_rate'] = df_tmp['amount_2019']/df_tmp['amount_all']

df_tmp.head(10)


# 参考：https://omathin.com/100knock-84/
# ただし，df_tmp_1, df_tmp_2とdf_customerとを結合する意味が解らなかった．

,customer_id,amount_2019,amount_all,amount_rate
0,CS001113000004,1298.0,1298,1.000000
1,CS001114000005,188.0,626,0.300319
2,CS001115000010,578.0,3044,0.189882
3,CS001205000004,702.0,1988,0.353119
4,CS001205000006,486.0,3337,0.145640
5,CS001211000025,456.0,456,1.000000
9,CS001212000070,456.0,456,1.000000
13,CS001214000009,664.0,4685,0.141729
14,CS001214000017,2962.0,4132,0.716844
16,CS001214000048,1889.0,2374,0.795703


In [ ]:
df_tmp.isnull().sum()

customer_id    0
amount_2019    0
amount_all     0
amount_rate    0
dtype: int64

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
df_customer_1 = pd.merge(df_customer[['customer_id', 'postal_cd']], df_geocode[['postal_cd', 'longitude', 'latitude']], on='postal_cd', how='inner')

# 「複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。」＝ 「customer_idごとにlongitutdeとlatitudeの平均をとる」
df_customer_1 = df_customer_1.groupby('customer_id').agg({'longitude':'mean', 'latitude':'mean'}).reset_index().rename(columns={'longitude':'mean_longitude', 'latitude':'mean_latitude'})
df_customer_1 = pd.merge(df_customer, df_customer_1, on='customer_id', how='inner')

df_customer_1.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,era,gender_era,mean_longitude,mean_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,30,31,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,60,69,139.83502,35.67193
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,40,41,139.68965,35.67374
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,60,61,139.48360,35.39125
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,20,21,139.70775,35.54084


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [ ]:
"""
問題点：application_store_cdに対応するカラム名がstore_cdとなっており，単純にmergeでon=XXとして結合できない
解決策：主に以下の2つ
  ① 無理やりmerge：今回の場合，left_on='application_store_cd', right_on='store_cd'とすることで，それらをキーとして結合する．
  ② リネーム：rename()を利用してカラム名を統一してあげる．
緯度：latitude
経度：longitude
"""
def calc_distance(from_longitude, from_latitude, to_longitude, to_latitude):
  phi1 = np.deg2rad(from_latitude)
  phi2 = np.deg2rad(to_latitude)
  ramda1 = np.deg2rad(from_longitude)
  ramda2 = np.deg2rad(to_longitude)

  sin_phi1 = np.sin(phi1)
  sin_phi2 = np.sin(phi2)
  cos_phi1 = np.cos(phi1)
  cos_phi2 = np.cos(phi2)
  cos_delta_ramda = np.cos(ramda1 - ramda2)

  return 6371*np.arccos(sin_phi1*sin_phi2 + cos_phi1*cos_phi2*cos_delta_ramda)
 
df_tmp = pd.merge(df_customer_1, df_store, how='inner', left_on='application_store_cd', right_on='store_cd')    
df_tmp['distance'] =   df_tmp[['mean_longitude', 'mean_latitude', 'longitude', 'latitude']].apply(lambda x: calc_distance(x[0], x[1], x[2], x[3]), axis=1)
df_tmp[['customer_id', 'address_x', 'address_y', 'distance']].head(10)


,customer_id,address_x,address_y,distance
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
1,CS021313000025,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282
2,CS021411000096,神奈川県伊勢原市高森**********,神奈川県伊勢原市伊勢原四丁目,2.480155
3,CS021415000150,神奈川県伊勢原市上粕屋**********,神奈川県伊勢原市伊勢原四丁目,2.734723
4,CS021313000046,神奈川県伊勢原市池端**********,神奈川県伊勢原市伊勢原四丁目,1.111911
5,CS021103000002,神奈川県伊勢原市西富岡**********,神奈川県伊勢原市伊勢原四丁目,2.384941
6,CS021214000028,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.399344
7,CS021512000095,神奈川県伊勢原市沼目**********,神奈川県伊勢原市伊勢原四丁目,1.993991
8,CS021613000002,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.399344
9,CS021412000147,神奈川県伊勢原市三ノ宮**********,神奈川県伊勢原市伊勢原四丁目,3.507680


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
"""
① 金額の合計を結合
② 金額の昇順，IDの降順に並べ替え
③ IDと住所が重複するものを削除
"""


df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_customer_u = pd.merge(df_customer, df_tmp, on='customer_id', how='left')    # innerにするとレコードの重複問題が消えてしまう

df_customer_u = df_customer_u.sort_values(['amount', 'customer_id'], ascending=[False, True])
# df.sort_values(['A', 'B'], ascending=[True, False])：'A'のTrueでソートして、その中で'B'のFalseでソートする
# sort_values('amount')：同一顧客に対しては売上金額合計が最も高いものを残す
# sort_values('customer_id')：売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残す

df_customer_u = df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'], keep='first')
# subset：デフォルトでは，すべてのカラムが一致しているときにdrop_duplicateされるが，
# keep=first：重複している物のうち，1番上に来ているものを残す．

print('df_customer:', len(df_customer),
      'df_customer_u:', len(df_customer_u),
      'diff:', len(df_customer) - len(df_customer_u))

df_customer_u.head()

df_customer: 21971 df_customer_u: 21941 diff: 30


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,era,gender_era,amount
16905,CS017415000097,福士 千夏,1,女性,1973-04-03,45,166-0014,東京都杉並区松ノ木**********,S13017,20151209,F-20101006-F,40,41,23086.0
12692,CS015415000185,岩淵 はるみ,1,女性,1973-09-19,45,135-0043,東京都江東区塩浜**********,S13015,20150322,F-20101014-F,40,41,20153.0
13550,CS031414000051,長澤 沙知絵,1,女性,1973-04-25,45,151-0064,東京都渋谷区上原**********,S13031,20150823,F-20101009-F,40,41,19202.0
4808,CS028415000007,紺野 あい,1,女性,1969-07-28,49,246-0023,神奈川県横浜市瀬谷区阿久和東**********,S14028,20151212,F-20100922-F,40,41,19127.0
14205,CS001605000009,安部 耕司,0,男性,1952-10-22,66,144-0035,東京都大田区南蒲田**********,S13001,20160203,F-20101019-E,60,60,18925.0


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [ ]:
df_customer_n = df_customer.copy()
df_customer_n = pd.merge(df_customer_n, df_customer_u[['amount', 'customer_id', 'customer_name', 'postal_cd']], on=['customer_name', 'postal_cd'], how='left')
df_customer_n = df_customer_n.rename(columns={'customer_id_x':'customer_id_default', 'customer_id_y':'customer_id_integrated'})
df_customer_n = df_customer_n.sort_values(['amount', 'customer_id_default'], ascending=[False, True]).drop_duplicates(subset=['customer_name', 'postal_cd'], keep='first')

df_customer_n['customer_id_default'] = df_customer_n['customer_id_integrated']
df_customer_n = df_customer_n.drop('customer_id_integrated', axis=1)
# df_tmp['integration_id'] = df_customer_n


df_customer_n.head()

,customer_id_default,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,era,gender_era,amount
16905,CS017415000097,福士 千夏,1,女性,1973-04-03,45,166-0014,東京都杉並区松ノ木**********,S13017,20151209,F-20101006-F,40,41,23086.0
12692,CS015415000185,岩淵 はるみ,1,女性,1973-09-19,45,135-0043,東京都江東区塩浜**********,S13015,20150322,F-20101014-F,40,41,20153.0
13550,CS031414000051,長澤 沙知絵,1,女性,1973-04-25,45,151-0064,東京都渋谷区上原**********,S13031,20150823,F-20101009-F,40,41,19202.0
4808,CS028415000007,紺野 あい,1,女性,1969-07-28,49,246-0023,神奈川県横浜市瀬谷区阿久和東**********,S14028,20151212,F-20100922-F,40,41,19127.0
14205,CS001605000009,安部 耕司,0,男性,1952-10-22,66,144-0035,東京都大田区南蒲田**********,S13001,20160203,F-20101019-E,60,60,18925.0


---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [ ]:
del df_customer_1
del df_customer_n

---
> P-089: 売上実績がある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
df_sales = df_receipt.groupby('customer_id').amount.sum().reset_index()

# df_salesとdf_customerをinnerでmergeすることで、「売り上げ実績のある顧客」だけを抽出
df_tmp = pd.merge(df_customer, df_sales, on='customer_id', how='inner')

df_train, df_test = train_test_split(df_tmp, train_size=0.8, random_state=100)


print(f'df_trainの割合：{len(df_train)/len(df_sales)}')
print(f'df_testの割合：{len(df_test)/len(df_sales)}')

df_trainの割合：0.7998073913566871
df_testの割合：0.20007222824124232


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [ ]:
a = 'abcdefg'
print(a[:-2])

abcde


In [ ]:
df_tmp = df_receipt.copy()
df_tmp['sales_ymd'] = df_tmp['sales_ymd'].astype('str')
df_tmp['sales_ym'] = df_tmp['sales_ymd'].apply(lambda x: x[:-2])

df_tmp = df_tmp.groupby('sales_ym').amount.sum().reset_index()

# def split_data(df, train_size, test_size, slide_window, start_point):
#     train_start = start_point * slide_window
#     test_start = train_start + train_size
#     return df[train_start : test_start], df[test_start : test_start + test_size]

# df_train_1, df_test_1 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=0)
# df_train_2, df_test_2 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=1)
# df_train_3, df_test_3 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=2)


def df_split(df, train_size, test_size, start_point):
  return df[start_point : (start_point + train_size)], df[(start_point + train_size) : (start_point + train_size + test_size)]

df_train_1, df_test_1 = df_split(df_tmp, train_size=12, test_size=6, start_point=0)
df_train_2, df_test_2 = df_split(df_tmp, train_size=12, test_size=6, start_point=6)
df_train_3, df_test_3 = df_split(df_tmp, train_size=12, test_size=6, start_point=12)


In [ ]:
df_train_1

,sales_ym,amount
0,201701,902056
1,201702,764413
2,201703,962945
3,201704,847566
4,201705,884010
5,201706,894242
6,201707,959205
7,201708,954836
8,201709,902037
9,201710,905739


In [ ]:
df_test_1

,sales_ym,amount
12,201801,944509
13,201802,864128
14,201803,946588
15,201804,937099
16,201805,1004438
17,201806,1012329


---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()    # 売り上げ実績がある顧客
df_tmp = pd.merge(df_tmp, df_customer, on='customer_id', how='right')

df_tmp['buy_flg'] = df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)
# 実績があれば1なければ0

rs = RandomUnderSampler(random_state=100)

print(f'flag==0の数：{len(df_tmp.query("buy_flg == 0"))}')
print(f'flag==1の数：{len(df_tmp.query("buy_flg == 1"))}')
print('----------------------')

df_tmp, _ = rs.fit_resample(df_tmp, df_tmp['buy_flg'])
# RandomUndrSample.fit_sample(X, y)：yの値が均等になるようにアンダーサンプリングがXに対して行われる。

print(f'flag==0の数：{len(df_tmp.query("buy_flg == 0"))}')
print(f'flag==1の数：{len(df_tmp.query("buy_flg == 1"))}')

flag==0の数：13665
flag==1の数：8306
----------------------
flag==0の数：8306
flag==1の数：8306


---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [ ]:
"""
① gender_cdとgenderは情報が完全一致しているので，これら2つを関連付けるdfを作成し，どちらか一方を削除してよい．
② df_customerから上述一報を削除したものを作成
③ df_customerを①と②に分割で完成
"""

df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_gender_s = df_customer.drop(columns='gender')

df_gender_s.head()

,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,era,gender_era
0,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,30,31
1,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,60,69
2,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,40,41
3,CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,60,61
4,CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,20,21


---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [ ]:
df_product_full = pd.merge(df_product, df_category[['category_small_cd', 'category_major_name', 'category_medium_name', 'category_small_name']], on='category_small_cd', how='inner')
"""
① df_product，df_categoryの共通カラムは[category_major_name, category_medium_name, category_small_name]
② これのうち，最も粒度が小さいのはcategory_small
③ ただし，category_small_nameは結合する必要があるから，onには，それに対応するcategory_small_cdを使う．
"""

df_product_full.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_product_full.to_csv('df_product_full_UTF-8_header.csv', encoding='utf8', header=True, index=False)
# df_product_fullにおけるindexは所詮数字なので，index=Falseとする．

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [ ]:
df_product_full.to_csv('df_product_full_CP932.csv', encoding='CP932', header=True, index=False)
# CP932：Shift_JISの友達

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_product_full.to_csv('df_product_full_UTF-8_nheader.csv', encoding='utf8', header=None, index=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭3件を表示させ、正しく取り込まれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp = pd.read_csv('df_product_full_UTF-8_header.csv', encoding='utf8')
df_tmp.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭3件を表示させ、正しく取り込まれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_tmp = pd.read_csv('df_product_full_UTF-8_nheader.csv', encoding='utf8')
df_tmp.head()

,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
0,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
1,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
2,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
3,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
4,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_product_full.to_csv('df_product_full_UTF-8_header.tsv', encoding='utf8')
# df.to_csv('*.tsv')でtsvを作成可能

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭3件を表示させ、正しく取り込まれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp = pd.read_csv('df_product_full_UTF-8_header.tsv', encoding='utf8')
df_tmp.head()

,Unnamed: 0,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！